In [37]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

'1.3.0'

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import torch
from models.gradientbased.MnistModels import MNISTmodel
torch.__version__

In [ ]:
activation_types = ["relu", "tanh", "sigmoid", "softplus"]
model_types = ["dnn", "cnn"]
args = dict(
    root = Path().home()/"data",
    project_path = Path().home()/"code"/"XAI",
    logterm = False, 
    sv_folder = "mnist", 
    n_step = 20,
    batch_size = 128,
    download = False,
    device = "cuda" if torch.cuda.is_available() else "cpu",
    seed = 73
)
# main(model_types, activation_types, **args)

In [61]:
model = model.cpu()
for imgs, labels in test_loader:
    break
    
num = 6
samples = imgs[:num].requires_grad_()
sample_labels = labels[:num]
model.zero_grad()
output = model(samples)
loss = loss_function(output, sample_labels)
loss.backward()
# input * grad
# inputgrads = samples * samples.grad
inputgrads = samples.grad

In [ ]:
fig, axes = plt.subplots(num, 2, figsize=(4, 12))
for i, (dual_ax, (sample, inputgrad), sample_label) in enumerate(zip(axes, list(zip(*[samples, inputgrads])), sample_labels)):
    # draw
    for ax, name in zip(dual_ax, ["sample", "input*grad"]):
        if name == "sample":
            img = sample
        else:
            img = inputgrad
        ax.imshow(img.permute(32, 32, -1).squeeze().detach())
        ax.grid(False)
        ax.axis("off")
        if i == 0:
            ax.set_title(f"{name}")
plt.tight_layout()
plt.show()